In [1]:
import requests
import pandas as pd
import pickle
from features.health_insurance import HealthInsurance
import json

In [2]:
df_raw = pd.read_csv('/home/ezequiel/Documentos/Comunidade_DS/health_insurance_cross_sell/data/raw/train.csv')

In [3]:
model = pickle.load(open('/home/ezequiel/Documentos/Comunidade_DS/health_insurance_cross_sell/models/linear_regression_model.pkl', 'rb'))

In [5]:
pipeline = HealthInsurance()

In [13]:
df_test = df_raw.copy()

In [43]:
df1 = HealthInsurance.columns_rename(pipeline, data=df_test)

In [45]:
df2 = HealthInsurance.feature_engineering(pipeline, data=df1)

In [46]:
df3 = HealthInsurance.data_preparation(pipeline, data=df2)

In [47]:
df_response = HealthInsurance.get_prediction(pipeline, model=model, original_data=df_raw, test_data=df3)

In [48]:
pd.DataFrame(json.loads(df_response)).head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,score
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1,0.302348
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0,0.035963
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1,0.300988
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0,0.000628
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0,0.000569


In [19]:
data = json.dumps(df_test.sample(500).to_dict(orient='records'))

In [20]:
df_test.shape

(381109, 12)

In [21]:
# API call
url = 'http://localhost:5000/src/models'
#header = {'Content-type': 'application/json'}
data = data

#r = requests.post(url, data=data, headers=header)
r = requests.post(url=url, json=data)
print(f'Status Code {r.status_code}')

Status Code 200


In [24]:
pd.json_normalize(r.json()).head(20).sort_values('score', ascending=False)

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response,score
8,232697,0.138038,0.076923,1,0.108925,0,between_1_2_years,1,-1.621992,26.0,0.100346,0,0.239959
4,132415,0.138038,0.461538,1,0.079868,0,between_1_2_years,1,0.623675,156.0,0.238754,1,0.232117
7,114643,0.138038,0.553846,1,0.116075,0,between_1_2_years,1,-1.621992,26.0,0.474048,0,0.228962
9,119510,0.138038,0.030769,1,0.086374,0,below_1_year,1,0.708910,26.0,0.809689,0,0.225830
10,196944,0.138038,0.630769,1,0.098092,0,between_1_2_years,1,0.888496,26.0,0.536332,0,0.215624
12,4595,0.104363,0.030769,1,0.126802,0,below_1_year,1,-0.342827,152.0,0.802768,0,0.214578
2,108014,0.104363,0.400000,1,0.070979,0,between_1_2_years,1,-1.621992,157.0,0.875433,0,0.203907
0,319901,0.104363,0.015385,1,0.103927,0,below_1_year,1,-0.372206,152.0,0.989619,1,0.197789
17,357301,0.104363,0.015385,1,0.098092,0,below_1_year,1,0.544362,152.0,0.737024,0,0.194951
15,47497,0.138038,0.600000,1,0.098092,0,between_1_2_years,0,0.244239,26.0,0.847751,0,0.033248
